In [ ]:
import json
import matplotlib.pyplot as plt

import numpy as np
import xgi
from src.GenerativeModels import uniform_planted_partition_hypergraph
from src.HypergraphContagion import Gillespie_SIS_two_communities

In [ ]:
rho = 0.506
epsilon2 = 0.18
epsilon3 = 0.95
num_realizations = 5

# parameters
n = 10000
k = 50
q = 50

# Epidemic parameters
gamma = 1
tmax = 300

beta2tilde = 0.2
beta3tilde = 4

rho1 = 0
rho2 = 1

In [ ]:
links = uniform_planted_partition_hypergraph(n, 2, k, epsilon2, rho)
triangles = uniform_planted_partition_hypergraph(n, 3, q, epsilon3, rho)
H = xgi.Hypergraph(links + triangles)

In [ ]:
community1 = {i for i in range(int(H.num_nodes * rho))}
community2 = {i for i in range(int(H.num_nodes * rho), n)}

beta2c = gamma / k
beta3c = gamma / q

beta2 = beta2tilde * beta2c
beta3 = beta3tilde * beta3c

beta = {2: beta2, 3: beta3}

In [ ]:
sims = dict()
for i in range(num_realizations):
    t, S1, I1, S2, I2 = Gillespie_SIS_two_communities(
        H,
        beta,
        gamma,
        community1,
        community2,
        rho1=rho1,
        rho2=rho2,
        tmin=0,
        tmax=300,
    )

    sims[i]["x1"] = list(I1/len(community1))
    sims[i]["x2"] = list(I2/len(community2))
    sims[i]["t"] = list(t)
    print(i)

In [ ]:
for i in range(num_realizations):
    plt.plot(sims[i]["x1"], sims[i]["x2"], linewidth=0.25)
plt.xlim([0, 1])
plt.ylim([0, 1])

In [ ]:
skip = 10
data = dict()
for i in range(num_realizations):
    data[f"{i}-x"] = list(zip(sims[0]["x1"][::skip], sims[0]["x2"][::skip]))
    data[f"{i}-t"] = sims[i]["t"][::skip]

In [ ]:
datastring = json.dumps(data)

with open(f"Data/polarization/{rho}-{epsilon2}-{epsilon3}-trajectories.json", "w") as output_file:
    output_file.write(datastring)